In [ ]:
!pip install plotly pandas geopandas scipy shapely numpy

# Visualize regions

In [ ]:
import json
import plotly.graph_objects as go
import pandas as pd
import geopandas as gpd
import plotly.express as px
from scipy.spatial import Voronoi, ConvexHull
import shapely.geometry as sg
import shapely.ops as so
import numpy as np

REGION_SHAPES_PATH = "./data/denmark-adm7.geojson"
METADATA_PATH = "./data/metadata-v2.json"

with open(METADATA_PATH, "r") as file:
    data = json.load(file)

data = [point for point in data.values() if point["cell"] != "None"]
df = pd.DataFrame(data)
df['point_size'] = 1


with open(REGION_SHAPES_PATH, "r") as file:
    shapes = json.load(file)

all_shapes = []
for shape in shapes["features"]:
    for polygon in shape["geometry"]["coordinates"]:
        polygon[0] = [(point[1], point[0]) for point in polygon[0]]
        all_shapes.append(sg.Polygon(polygon[0]))

all_shapes = so.unary_union(all_shapes)
denmark_shape = []
# Remove holes from the shape
for shape in all_shapes.geoms:
    denmark_shape.append(sg.Polygon(shape.exterior))

denmark_shape = so.unary_union(denmark_shape)


def create_convex_hull_polygon(points):
    # Convert data to coordinates
    coordinates = np.array([(point["lat"], point["lng"]) for point in points])

    # Compute convex hull
    hull = ConvexHull(coordinates)

    # Extract x and y coordinates for the convex hull
    x = [coordinates[j, 0] for j in hull.vertices]
    y = [coordinates[j, 1] for j in hull.vertices]

    # Create a Shapely polygon for the convex hull
    convex_hull_polygon = sg.Polygon(list(zip(x, y)))

    return convex_hull_polygon


def compute_voronoi_cells(points):
    coordinates = np.array([(point["lat"], point["lng"]) for point in points])
    vor = Voronoi(coordinates)

    voronoi_cells = {}
    all_cells = []
    single_points = {}

    for i, point in enumerate(data):
        region = vor.regions[vor.point_region[i]]
        cell_type = point["cell"]

        if len(region) == 0:
            continue
        if -1 in region:
            new_point = (point["lat"], point["lng"])
            if cell_type in single_points:
                single_points[cell_type].append(new_point)
            else:
                single_points[cell_type] = [new_point]
            continue
        # Extract x and y coordinates for the region
        x = [vor.vertices[j, 0] for j in region]
        y = [vor.vertices[j, 1] for j in region]

        # Create a list of coordinates for the cell
        cell_coordinates = list(zip(x, y))
        polygon = sg.Polygon(cell_coordinates)
        polygon = polygon.intersection(denmark_shape)
        # Aggregate coordinates for each cell_type
        if cell_type in voronoi_cells:
            voronoi_cells[cell_type].append(polygon)
        else:
            voronoi_cells[cell_type] = [polygon]

        all_cells.append(sg.Polygon(cell_coordinates))
    combined_polygons = {cell_type: so.unary_union(polygons) for cell_type, polygons in voronoi_cells.items()}
    for cell_type, polygon in combined_polygons.items():
        if isinstance(polygon, sg.MultiPolygon):
            print(cell_type)
    return combined_polygons, all_cells


all_points_boundary = create_convex_hull_polygon(data)
voronoi_polygons, all_cells = compute_voronoi_cells(data)

# Create a GeoDataFrame for the Voronoi polygons
voronoi_gdf = gpd.GeoDataFrame(geometry=list(voronoi_polygons.values()))

# Create a GeoDataFrame for the original data points
points_gdf = gpd.GeoDataFrame(geometry=[sg.Point(point["lat"], point["lng"]) for point in data])
points_gdf["size"] = 1
points_gdf["color"] = [point["cell"] for point in data]

# Plot using px.scatter_mapbox
fig = px.scatter_mapbox(points_gdf,
                        lat=points_gdf.geometry.x,
                        lon=points_gdf.geometry.y,
                        mapbox_style="carto-positron",
                        title='Voronoi Polygons with Convex Hull',
                        size="size",
                        color="color",
                        size_max=4,
                        opacity=0.5
                        )

# Add convex hull and Voronoi polygons to the plot
fig.update_geos(fitbounds="locations", visible=False)


def draw_polygon(polygon):
    # Count number of points from data in the polygon
    num_points = points_gdf.within(polygon).sum()
    if num_points > 8:
        fig.add_trace(go.Scattermapbox(
            lat=polygon.exterior.xy[0].tolist(),
            lon=polygon.exterior.xy[1].tolist(),
            mode='lines',
            fill='none',
            name='Voronoi Polygon',
            line=dict(color='blue', width=2),
            showlegend=False,
        ))
    else:
        print("skip")


for geometry in voronoi_gdf.geometry:
    # Check if it's a Polygon or MultiPolygon
    if geometry.geom_type == 'Polygon':
        # For a single Polygon, add the trace directly
        draw_polygon(geometry)
    elif geometry.geom_type == 'MultiPolygon':
        # For a MultiPolygon, iterate over its constituent polygons and add traces for each
        for polygon in geometry.geoms:
            draw_polygon(polygon)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()